In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os
import dpsimpy
import glob
import requests

%matplotlib widget

In [ ]:
example_name = "DP_WSCC9bus_SGReducedOrderVBR"

final_time = 2.0
time_step = 100e-6
with_fault = True
start_time_fault = 1.2
end_time_fault = 1.3

sg_type = "4"
fault_bus_name = "BUS6"
log_dir = "logs"

log_level = dpsimpy.LogLevel.info

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

filename = 'WSCC-09_Dyn_Fourth'

url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/WSCC-09/WSCC-09_Dyn_Fourth/WSCC-09_Dyn_Fourth'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')
download_grid_data(filename+'_DI.xml', url+'_DI.xml')
    
files = glob.glob(filename+'_*.xml')

In [ ]:
## Powerflow Simulation
sim_name_pf = example_name + "_PF"

dpsimpy.Logger.set_log_dir(log_dir+"/"+sim_name_pf)
reader = dpsimpy.CIMReader(sim_name_pf, log_level, log_level)

system_pf = reader.loadCIM(60, files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)
system_pf.component('GEN1').modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)
logger_pf = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger_pf.log_attribute(node.name() + ".V", node.attr("v"))
    
sim_pf = dpsimpy.Simulation(sim_name_pf, log_level)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(final_time)
sim_pf.set_final_time(2*final_time)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.do_init_from_nodes_and_terminals(True)
sim_pf.add_logger(logger_pf)
sim_pf.run()

In [ ]:
## Dynamic Simulation
def run_simulation(sim_name, implementation):
   
    dpsimpy.Logger.set_log_dir(log_dir+"/"+sim_name)
    reader2 = dpsimpy.CIMReader(sim_name, log_level, log_level)
    if sg_type == "3":
        sys = reader2.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.SG3OrderVBR)
    elif sg_type == "4":
        sys = reader2.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.SG4OrderVBR)
    elif sg_type == "6b":
        sys = reader2.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.SG6bOrderVBR)
    else:
        raise Exception("Unsupported reduced-order SG type!")

    fault_dp = dpsimpy.sp.ph1.Switch("Fault", log_level)

    if with_fault == True:
        n1_dp = dpsimpy.dp.SimNode(fault_bus_name, dpsimpy.PhaseType.Single)
        fault_dp.set_parameters(1e12, 0.02*529)
        fault_dp.connect([dpsimpy.dp.SimNode.gnd, n1_dp])
        fault_dp.open()
        sys.add(fault_dp)

    sys.init_with_powerflow(system_pf)
    
    for comp in sys.components:
       if comp.__class__ == dpsimpy.dp.ph1.SynchronGenerator4OrderVBR:
           gen_pf = system_pf.component(comp.name())
           comp.get_terminal(index=0).set_power(- gen_pf.get_apparent_power())
           comp.set_model_as_norton_source(False)

    logger = dpsimpy.Logger(sim_name)
    for node in sys.nodes:
        logger.log_attribute(node.name() + ".V", node.attr("v"))

    for comp in sys.components:
        if comp.__class__ == dpsimpy.dp.ph1.SynchronGenerator4OrderVBR:
            logger.log_attribute(comp.name() + ".Tm", comp.attr("Tm"))
            logger.log_attribute(comp.name() + ".Te", comp.attr("Te"))
            logger.log_attribute(comp.name() + ".omega", comp.attr("w_r"))
            logger.log_attribute(comp.name() + ".delta", comp.attr("delta"))

    sim = dpsimpy.Simulation(sim_name, log_level)
    sim.set_system(sys)
    sim.set_domain(dpsimpy.Domain.DP)
    sim.set_solver(dpsimpy.Solver.MNA)
    sim.set_time_step(time_step)
    sim.set_final_time(final_time)
    sim.do_system_matrix_recomputation(True)
    sim.set_direct_solver_implementation(implementation)
    sim.add_logger(logger)

    if with_fault:
        fault_event1 = dpsimpy.event.SwitchEvent(start_time_fault, fault_dp, True)
        fault_event2 = dpsimpy.event.SwitchEvent(end_time_fault, fault_dp, False)
        sim.add_event(fault_event1)
        sim.add_event(fault_event2)

    sim.run()

In [ ]:
%%capture
sim_name = example_name+'_SparseLU'
run_simulation(sim_name,dpsimpy.DirectLinearSolverImpl.SparseLU)
ts_dpsim_sparse = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+".csv")

In [ ]:
%%capture
sim_name = example_name+'_KLU'
run_simulation(example_name+'_KLU',dpsimpy.DirectLinearSolverImpl.KLU)
ts_dpsim_klu = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+".csv")

In [ ]:
var_name = 'GEN3.Te'
plt.figure()
plt.plot(ts_dpsim_sparse[var_name].time, ts_dpsim_sparse[var_name].values)
plt.plot(ts_dpsim_klu[var_name].time, ts_dpsim_klu[var_name].values, linestyle='--')